<br>

<h1>
    <center>
        Estimating $\pi$ using Various Methods
    </center>
</h1>

## Table of Contents


0. [Define Problem](#0.-Define-Problem)
1. [Import Libraries](#1.-Import-Libraries)
2. [Estimate $\pi$ using Taylor Series Expansion](#2.-Estimate-$\pi$-using-Taylor-Series-Expansion) 
    * 2.1 Define Functions
        * GenerateTaylorSeries
    * 2.2 Perform Computations
    * 2.3 Construct Records DataFrame
3. [Estimate $\pi$ using Monte Carlo Simulation with Numpy](#3.-Estimate-$\pi$-using-Monte-Carlo-Simulation-with-Numpy)
    * 3.1 Perform Numpy Computations
    * 3.2 Construct Records DataFrame
4. [Estimate $\pi$ using Quantum Monte Carlo Simulation](#4.-Estimate-$\pi$-using-Quantum-Monte-Carlo-Simulation) 
    * 4.1 Define Functions
        * CreateCircuit
        * GenerateRandomNumbers_QC
        * NormalizeNumbers
    * 4.2 Perform Computations
    * 4.3 Construct Records DataFrame
5. [Analysis of Performance](#5.-Analysis-of-Performance)
    * 5.1 Construct $\pi$ Approximation Performance DataFrame
    * 5.2 Plot Sample Size vs. $\pi$ Approximation
    * 5.3 Plot Sample Size vs. $\pi$ Approximation Error
    * 5.4 Plot Computational Time vs. $\pi$ Approximation Error
6. [Conclusion - Which is better?](#6.-Conclusion)

### 0. Define Problem

<br>

The below code is dedicated to approximating $\pi$ using various computational methods. The famously irrational number can be approximated using iterative computational methods envolving both but not excluding the __Taylor Series__ approximation and __Monte Carlo__ simulation methods. As previously mentioned, each of the respective methods is reliant on the use of iterative computats or as the following code shall refferences it as _population_. Each iteration involves different population sizes ranging in value of $10^{1}, 10^{2}, 10^{3}, 10^{4}, 10^{5}, 10^{6}, 10^{7}, 10^{8}$ and each of the successive iterations in each of the aforementioned populations should more closely approximate $\pi$. However dependening on the method used, the difference between the true value of $\pi and its approximated value should become smaller, faster with successive iterations. The below code will attempt to measure which method is better in vaim metrcis associated accuracy and speed.


<br>

#### Approximating $\pi$ vai the Taylor Series Expantion:

$$\pi = 4 \sum_{i=1}^{n} \frac{(-1)^{n}}{2n-1}$$

$$ \pi = 4 \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} ... \frac{1}{n} \right) = 3.1415926...$$

<br>

#### Aproximate $\pi$ via Monte Carlo Simulation:


$$ \pi \sim \frac{ || \sqrt{(x^{2} + y^{2})} || < 1 }{n}$$

<br>

![MC_Simulation](Monte_Carlo_Approximation.gif)

### 1. Import Libraries

In [1]:
# import visulaization tool
import seaborn as sns
import matplotlib.pyplot as plt

# import numerical processing tool
import numpy as np
np.random.seed(1234)

# data handeling
import pandas as pd

# measure time/duration 
import time

# import quantum simulation tools
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import BasicAer, execute
import pylatexenc

### 2. Estimate $\pi$ using Taylor Series Expansion

$$\pi = 4 \sum_{i=1}^{n} \frac{(-1)^{n}}{2n-1} = 4  \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} ... \frac{1}{n }\right)$$


#### 2.1 Define Functions - GenerateTaylorSeries

In [2]:
def GenerateTaylorSeries(x):
    """
    Input:
        x | int | numebr of iterations to approximate taylor series expansion pi/4
    Output:
        pi_aproximation | float | approximation of pi        
    """
        
    iteration_value = [(1**(2*i+1)*(-1)**i)/(2*i+1) for i in np.arange(x)]
    summation = np.sum(iteration_value)
    pi_approximation = 4 * summation
        
    return pi_approximation

#### 2.2 Perform Computations


In [3]:
# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate taylor series
    pi_approximation = GenerateTaylorSeries(x = sample_size)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)
    

100
1000
10000
100000
1000000
10000000
100000000


#### 2.3 Construct Records DataFrame

In [4]:
TaylorSeries_Performance = (
    pd
    .DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(TaylorSeries_Performance)

,pi_approximatation,population,duration,error
0,3.131593,100,0.000993,-9.999750e-03
1,3.140593,1000,0.000000,-9.999998e-04
2,3.141493,10000,0.006511,-1.000000e-04
3,3.141583,100000,0.073933,-1.000000e-05
4,3.141592,1000000,0.773516,-1.000000e-06
5,3.141593,10000000,7.815977,-1.000000e-07
6,3.141593,100000000,79.414183,-9.999976e-09


### 3. Estimate $\pi$ using Monte Carlo Simulation with Numpy

#### 3.1 Perform Numpy Computations

In [5]:
# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate x values
    x = np.random.random_sample(sample_size)
    # generate y values
    y = np.random.random_sample(sample_size)
    
    # create MC 
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values <= 1]) / len(x)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)

100
1000
10000
100000
1000000
10000000
100000000


#### 3.2 Construct Records DataFrame

In [6]:
Numpy_Performance = (
    pd
    .DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(Numpy_Performance)

,pi_approximatation,population,duration,error
0,3.120000,100,0.000000,-0.021593
1,3.088000,1000,0.000993,-0.053593
2,3.132400,10000,0.000000,-0.009193
3,3.145800,100000,0.002990,0.004207
4,3.140820,1000000,0.023587,-0.000773
5,3.141036,10000000,0.233634,-0.000556
6,3.141538,100000000,2.479883,-0.000054


### 4. Estimate $\pi$ using Quantum Monte Carlo Simulation

#### 4.1 Define Functions - CreateCircuit

In [7]:
def CreateCircuit(bits):
    """
    Input:
        bits | int | defines how many q-bits the circuit involves
    Output:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        circuit_diagram | matplotlib.figure | figure that depicts the q-bit to c-bit figure
    """    
    
    # define quantum register | classical register
    q = QuantumRegister(bits);  c = ClassicalRegister(bits)
    
    # define circuit
    circuit = QuantumCircuit(q, c)
    
    # use for loops to add q-bits and hadamar gates to the existing circuit
    for j in range(bits): circuit.h(q[j])
    
    # add measurements to transfer from q-bit to classical-bit
    circuit.measure(q, c)
    
    # draw circuit
    circuit_drawing = circuit.draw(output='mpl')
    
    # return circuit and drawing
    #return (circuit, circuit_drawing)
    return circuit

#### 4.1 Define Functions - GenerateRandomNumbers_QC

In [8]:
def GenerateRandomNumbers_QC(circuit, sample_size):
    """
    Input:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        sample_size | int | number of iterations to generate 8-bit numbers
    Output:
        numbers | list(int) | list of binary numbers generated by the circuit
    """   
    
    # generate binary numbers
    binary_numbers = execute(
        experiments = circuit,
        backend = BasicAer.get_backend('qasm_simulator'),
        shots = sample_size,
        memory = True
    )
    
    # convert binary numbers to integers
    numbers = np.array([int(i, 2) for i in binary_numbers.result().get_memory()])
    
    return numbers

#### 4.1 Define Functions - NormalizeNumbers

In [9]:
def NormalizeNumbers(numbers):
    """
    Input:
        numbers | list(int) | list of binary numbers generated by the circuit
    Output:
        normalized_values | list(float) | list of normalized numbers between [0,1]
    """
    minimum = min(numbers)
    maximum = max(numbers)
    
    normalized_values = np.array([(num - maximum) / (maximum - minimum) for num in numbers])
    
    return normalized_values

#### 4.2 Perform Computations

In [ ]:
# establish circute
bits = 8
circuit = CreateCircuit(bits = bits)

# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

# perform iterations    
for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate x values
    x = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    # generate y values
    y = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    
    # create MC
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values <= 1]) / len(x)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)

100
1000
10000
100000
1000000
10000000
100000000


#### 4.3 Construct Records DataFrame

In [ ]:
QuantumComputing_Performance = (
    pd
    .DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(QuantumComputing_Performance)

### 5. Analysis of Performance

#### 5.1 Construct $\pi$ Approximation Performance DataFrame

In [ ]:
TaylorSeries_Performance = TaylorSeries_Performance.assign(method = 'TaylorSeries')
Numpy_Performance = Numpy_Performance.assign(method = 'NumpyMC')
QuantumComputing_Performance = QuantumComputing_Performance.assign(method = 'QCMC')

Performance_DataFrame = (
    pd
    .concat(
        [TaylorSeries_Performance, Numpy_Performance, QuantumComputing_Performance]
    )
    [['method', 'population', 'duration', 'pi_approximatation', 'error']]
)

#### 5.2 Plot Sample Size vs. $\pi$ Approximation

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(log_pop = lambda x: np.log10(x['population']))
        .plot(
            x = 'log_pop',
            y = 'pi_approximatation',
            ax = ax0,
            style = "o--",
            markersize = 10,            
            xlabel = "log10 Computational Iterations",
            ylabel = "Approximation to Pi",
            title = "Iterative Approximation of Pi",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = np.pi,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.3 Plot Sample Size vs. $\pi$ Approximation Error

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(log_pop = lambda x: np.log10(x['population']))
        .plot(
            x = 'log_pop',
            y = 'error',
            ax = ax0,
            style = "o--",
            markersize = 10,            
            xlabel = "log10 Computational Iterations",
            ylabel = "Error",
            title = "Error Approximation of Pi",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = 0,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.4 Plot Computational Time vs. $\pi$ Approximation Error

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(
            log_pop = lambda x: np.log10(x['population']),
            log_duration = lambda x: np.log(x['duration'])
        )
        .sort_values('log_duration')
        .plot(
            x = 'log_duration',
            y = 'error',
            ax = ax0,
            alpha = 0.6,
            style = "o--",            
            markersize = 10,
            xlabel = "log10 Computational Duration log10(seconds)",
            ylabel = "Error Approximation to Pi",
            title = "Duration anad Error Pi",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = 0,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

### 6. Conclusion